# Extract and Transform Data From Database

### Step 1 - Running Database (Postgres) with Docker

```bash
docker run --env=POSTGRES_PASSWORD=devpass --env=POSTGRES_USER=etl --env=POSTGRES_DB=etl -p 5432:5432 -d postgres:12.16
```

### Step 2 - Install and Import postgres library

In [ ]:
# Only need to run it once for the whole project
!pip install psycopg2-binary

In [ ]:
import psycopg2

In [ ]:
import os

# make directory generated for generated data
os.makedirs("generated", exist_ok=True)

### Step 3 - Connect to Postgres Database

In [ ]:
# create a connection to postgres database
conn = psycopg2.connect(
    host="172.17.0.1",
    database="etl",
    user="etl",
    password="devpass",
    port=5432
)

### Step 4 - Initializing Mock Database (Optional)

In [ ]:
# create an exam_grades table if missing
with conn.cursor() as cur:
    cur.execute("""
CREATE TABLE IF NOT EXISTS "public"."exam_grades" (
  "semester" varchar(255),
  "sex" varchar(255),
  "exam1" numeric,
  "exam2" numeric,
  "exam3" numeric,
  "course_grade" numeric
)""")
    conn.commit()

In [ ]:
# load data from csv to database
with conn.cursor() as cur:
    with open("data/exam_grades.csv", "r") as f:
        # skip header
        next(f)
        cur.copy_from(f, "exam_grades", sep=",")
        conn.commit()

### Step 5 - Reading Data and Write to CSV (Extract but No Transform)

In [ ]:
# select all rows from exam_grades table and export to csv
with conn.cursor() as cur:
    cur.execute("SELECT * FROM exam_grades")
    with open("generated/exam_grades-from-db.csv", "w") as f:
        # write csv header
        f.write("semester,sex,exam1,exam2,exam3,course_grade\n")
        for row in cur:
            row = list(row)
            f.write(",".join(str(cell) for cell in row) + "\n")


### Step 6 - Reading Data and Write to CSV (Extract and Transform)

In [ ]:
# Extract and Transform
with conn.cursor() as cur:
    cur.execute("SELECT * FROM exam_grades")
    with open("generated/exam_grades-from-db-tf.csv", "w") as f:
        # write csv header
        f.write("year,term,gender,exam1,exam2,exam3,course_grade\n")
        for row in cur:
            row = list(row)
            semester = row[0]
            sex = row[1]
            exam1 = row[2]
            exam2 = row[3]
            exam3 = row[4]
            course_grade = row[5]

            year, term = semester.split("-", 1)
            gender = "Male" if sex == "Man" else "Female"
            new_row = [year, term, gender, exam1, exam2, exam3, course_grade]
            f.write(",".join(str(cell) for cell in new_row) + "\n")


### Step 7 - Close the Connection to Postgres Database

In [ ]:
conn.close()